In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

CONTINENTS = ['Africa', 'Asia', 'Oceania', 'Europe', 'Northern America', 'South America', 'Antarctic Region']
TOP_PRODUCERS = ['United States of America',
 'India',
 'China',
 'Brazil',
 'Russian Federation',
 'Canada',
 'Indonesia',
 'Ethiopia',
 'Democratic Republic of the Congo',
 'Nigeria',
 'Sweden',
 'Chile',
 'Finland',
 'Germany',
 'France',
 'Uganda',
 'Ghana',
 'Mexico',
 'Myanmar',
 'Poland',
 'Viet Nam',
 'Pakistan',
 'Thailand',
 'Australia',
 'New Zealand',
 'Kenya',
 'South Africa',
 'United Republic of Tanzania',
 'Bangladesh',
 'Turkey']

In [2]:
FAOSTAT_country_data = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/FAOSTAT_countries.csv", engine='python')

In [3]:
FAOSTAT_country_data = FAOSTAT_country_data[FAOSTAT_country_data.Country.isin(TOP_PRODUCERS) & FAOSTAT_country_data["Country Group"].isin(CONTINENTS)]

In [4]:
FAOSTAT_country_data = FAOSTAT_country_data[["Country", "Country Group"]]

In [5]:
# load datasets
forest_all = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Forestry_E_All_Data_cleared.csv", engine='python')

land = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Inputs_LandUse_E_All_Data_cleared.csv",engine='python')

In [6]:
forest_area = land[land.Item.isin(['Forestry']) & 
                   land.Area.isin(TOP_PRODUCERS) & 
                   land.Year.isin([1995, 2015])][["Area", "Year", "Unit", "Value", "Item"]]

In [7]:
country_area = land[land.Item.isin(['Country area']) & 
                   land.Area.isin(TOP_PRODUCERS) & 
                   land.Year.isin([2015])][["Area", "Year", "Unit", "Value", "Item"]]

In [8]:
forest_area.loc[forest_area.Year == 1995, 'Value'] = forest_area[forest_area.Year == 1995].Value.apply(lambda x: x*-1)


In [9]:
forest_area = forest_area.groupby(["Area"]).Value.sum().reset_index()

In [10]:
forest_area = forest_area.merge(FAOSTAT_country_data,left_on="Area", right_on="Country")

In [11]:
forest_area = forest_area.merge(country_area[["Area", "Value"]], left_on=["Area"], right_on=["Area"], suffixes=('_forest', '_country'))
 

In [12]:
forest_area["Decrease"] = forest_area["Value_forest"] / forest_area["Value_country"] 

In [13]:
forest_area = forest_area[["Area", "Decrease", "Country Group"]]

In [31]:
forest_area = forest_area.sort_values(by="Decrease")

In [32]:
forest_area.Area.unique()

array(['Myanmar', 'Indonesia', 'Uganda', 'Nigeria',
       'United Republic of Tanzania', 'Brazil',
       'Democratic Republic of the Congo', 'Ethiopia', 'Pakistan',
       'Australia', 'Bangladesh', 'Canada', 'Sweden', 'South Africa',
       'Finland', 'Germany', 'Russian Federation', 'Kenya',
       'United States of America', 'New Zealand', 'Poland', 'Thailand',
       'India', 'Turkey', 'Ghana', 'Chile', 'France', 'Viet Nam'],
      dtype=object)

In [33]:
import plotly.express as px


fig = px.bar(forest_area, x='Area', y='Decrease', color='Country Group',
             labels={'Decrease':'Increase of forested area since 1995 (%)', 'Area': 'Country', 'Country Group': "Continent"})

fig.update_layout(
    showlegend=True,
    xaxis_title=None,
    width=700,
    height=800,
) 

fig.show()

## Wood producers and deforestation

All of therese countries lied in the production of roundwood in the world, but the effects of wood logging on the forested area are diferent in every country. All of the european natios were able not to decrease share of the forrested areas even trough the fact that this group of 6 countries produces 13% of the world roundwood. The complete opposit trend can be observed in Africa where in some of the countires forsted are decreased by more than 8% in last 20 years. 

In [34]:
# export the plot to html
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import plotly.graph_objs as go
from IPython.core.display import display, HTML

plot(fig, filename = '3_production_deforestation.html')


'3_production_deforestation.html'